In [1]:
# Import dependencies
from config import genius_token
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
base_url = 'http://api.genius.com'
headers = {'Authorization': f'Bearer {genius_token}'}

In [3]:
# Create DataFrame from Spotify API calls
song_artist_category_df = pd.read_csv('../../preliminary_dataframes/song_artist_category3.csv')
song_artist_category_df.head()

,song,artist,artist_id,category,genres,filterd_genres
0,positions,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['dance', 'pop', 'post-teen']","['dance', 'pop']"
1,Levitating (feat. DaBaby),Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['uk', 'dance', 'pop']","['uk', 'dance', 'pop']"
2,Therefore I Am,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop']
3,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['dance', 'viral', 'pop', 'post-teen', 'canadi...","['dance', 'viral', 'pop', 'canadian']"
4,HOLIDAY,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['queer', 'lgbtq+', 'pop', 'country']","['pop', 'country']"


In [ ]:
# Create a list of lyrics for the first 100 songs
lyrics_list = []
t0 = time.time()
for index, row in song_artist_category_df[0:100].iterrows():
    print(index, row['song'], row['artist'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        found_song = False
        for id, hit in enumerate(results['response']['hits']):
            full_title = hit['result']['full_title'].lower()
            if artist.lower() in full_title:
                #print(full_title)
                found_song = True
                api_path = hit['result']['api_path']
                song_url = base_url + api_path
                response = requests.get(song_url, headers=headers).json()
                path = response['response']['song']['path']
                page_url = 'http://genius.com' + path
                page = requests.get(page_url)
                html = BeautifulSoup(page.text, 'html.parser')
                # Remove script tags from the lyrics
                [h.extract() for h in html('script')]
                lyrics = html.find('div', class_='lyrics').get_text()
                lyrics_list.append(lyrics)
                break
        if found_song == False:
            lyrics_list.append(None)
            print('Song not found.')
    except: 
        lyrics_list.append(None)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

0 positions Ariana Grande
2 Therefore I Am Billie Eilish
3 Monster (Shawn Mendes & Justin Bieber) Shawn Mendes
4 HOLIDAY Lil Nas X
5 On Me Lil Baby
Song not found.
6 Tyler Herro Jack Harlow
7 Drankin N Smokin Future
8 Whoopty CJ
9 Errbody Lil Baby
Song not found.
10 Regardless RAYE
11 Save My Life (feat. Lovespeake) David Guetta
Song not found.
12 Paul Is Dead Scooter
13 No Time For Tears Nathan Dawe
14 Lovefool (feat. Pia Mia) twocolors
16 Dirty grandson
18 Trouble’s Coming Royal Blood
Song not found.
19 Take Me Away Ayron Jones
Song not found.
20 Not Another Love Song Ella Mai
Song not found.
21 Damage H.E.R.
22 Hit Different SZA
23 Wake Up Love Teyana Taylor
Song not found.
24 Pick Up Your Feelings Jazmine Sullivan
Song not found.
25 Blinding Lights The Weeknd
Song not found.
26 Dance Monkey Tones And I
27 The Box Roddy Ricch
28 Roses - Imanbek Remix SAINt JHN
Song not found.
29 Don't Start Now Dua Lipa
Song not found.
30 Dakiti Bad Bunny


In [44]:
# Create a list of lyrics for the first 100 songs
lyrics_list = []
t0 = time.time()
for index, row in song_artist_category_df[0:100].iterrows():
    print(index, row['song'], row['artist'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            print(full_title)
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

0 positions Ariana Grande
positions by Ariana Grande
2 Therefore I Am Billie Eilish
Therefore I Am by Billie Eilish
3 Monster (Shawn Mendes & Justin Bieber) Shawn Mendes
Monster by Shawn Mendes & Justin Bieber
4 HOLIDAY Lil Nas X
HOLIDAY by Lil Nas X
5 On Me Lil Baby


KeyboardInterrupt: 

In [5]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[100:200].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            #print(full_title)
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

100 All I Want for Christmas Is You
101 Last Christmas
102 Santa Tell Me
103 Rockin' Around The Christmas Tree - Single Version
Song not found.
104 It's Beginning to Look a Lot like Christmas
105 Chanukah Song - The
106 Happy Hanukkah
107 Oh Hanukkah
108 Maoz Tzur
109 If It Be Your Will
110 A Holly Jolly Christmas - Single Version
Song not found.
111 Christmas (Baby Please Come Home)
112 It's Beginning to Look a Lot Like Christmas (with Mitchell Ayres & His Orchestra)
113 Jingle Bell Rock
114 Rockin' Around The Christmas Tree
115 Santa Tell Me
116 All I Want for Christmas Is You
117 Mistletoe
118 Holly Jolly Christmas
119 Naughty List (with Dixie D’Amelio)
120 hole in the bottle (with Shania Twain)
121 One Of Them Girls
122 Down To One
123 GOOD TIME
124 Forever After All
125 She Don't Know She's Beautiful
126 When You Say Nothing At All
127 Bye-Bye
128 Neon Moon
129 Just To See You Smile
130 Forever After All
131 The Bones
132 The Good Ones
133 Got What I Got
134 Love You Like I Used T

200

In [6]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[200:300].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

200 TE MUDASTE
201 BICHOTA
202 La Luz
203 Ponte Pa' Mi
204 La Nota
205 Un Amor Eterno - Versión Balada
206 Hoy Tengo Ganas De Ti
207 Corazón partío
208 Dígale
209 Completamente Enamorados
210 Blinding Lights (with ROSALÍA) - Remix
Song not found.
211 Lento
212 Más De La Una
213 Te Olvidare
214 No Me Acostumbro
215 s a l u d o s
216 Lotería
Song not found.
217 Qué Vas A Hacer?
Song not found.
218 Te bloqueo
219 Vámonos
220 Levitating (feat. DaBaby)
Song not found.
221 The Best Days (feat. Tabitha)
222 Love Not War (The Tampa Beat)
223 Edge of Midnight (Midnight Sky Remix) (feat. Stevie Nicks)
Song not found.
224 Fly Away
225 This Will Be (An Everlasting Love)
226 Midnight Train to Georgia
227 How Long Do I Have to Wait for You?
228 Are You Ready to Love Me?
229 Love Train
230 Garden Song
231 Ain't No Grave (Sparse)
Song not found.
232 Washed Away
233 Farm
234 Drawing out the Line
235 Kiriume
236 Beginning
Song not found.
237 Morgensol
No result found.
238 Con el tiempo todo se consigue


300

In [7]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[300:400].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

300 Remembering
Song not found.
301 Reveries
No result found.
302 Sea Change
No result found.
303 All that Glitters
304 Vals De Rodrigues
305 Stardust
No result found.
306 May
307 Oltre - The Shape Of Piano To Come Vol. I
No result found.
308 Gaia - The Shape Of Piano To Come Vol. I
No result found.
309 Mare di dicembre
No result found.
310 Remembering
Song not found.
311 Rites of Passage
Song not found.
312 Things Unsaid
No result found.
313 Gouttes
Song not found.
314 silbermond
No result found.
315 Young Love
Song not found.
316 Now and Then
Song not found.
317 Memories In Red
Song not found.
318 Asilah
Song not found.
319 Humbly
Song not found.
320 Regardless
321 Save My Life (feat. Lovespeake)
322 Paul Is Dead
323 No Time For Tears
324 Lovefool (feat. Pia Mia)
325 don't be so hard on yourself
Song not found.
326 All My Days
327 Her Theme
Song not found.
328 I Can't See
Song not found.
329 Ikebana
No result found.
330 Beirut
331 Kings
332 TIKTOK
333 Operator Bludgeoning
Song not fo

400

In [8]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[400:500].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

400 Sexy Boy
401 You Can Have It All
402 The First Big Weekend
403 Girl From Mars
404 New French Girlfriend
405 Nothing To Be Done
406 Alison
407 Call Me
408 Just Like Honey
409 Genius of Love
410 Close To Me
411 Blue Monday - 2016 Remaster
Song not found.
412 This Charming Man - 2011 Remaster
Song not found.
413 Town Called Malice
414 She Sells Sanctuary
415 Grass
416 Hope There's Someone
417 Wake Up
418 I Bet You Look Good On The Dancefloor
419 Killamangiro
420 Like You Love Me
421 Pilots
Song not found.
422 LET'S STAY HOME TONIGHT
423 You
424 Can't Quit You
425 The War We Made
426 Save Me
427 Alive Again
428 What Are You Waiting For
Song not found.
429 I Won't Look Back
430 Dreams
431 Rest
Song not found.
432 Kani (Day)
No result found.
433 Brave New World (Instrumental)
434 Come Near
No result found.
435 Go Go Go
Song not found.
436 Mark My Hands
Song not found.
437 Alchemy
Song not found.
438 Collide
Song not found.
439 Queen of a Wasteland
440 Another In The Fire - Live
441 What 

500

In [9]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[500:600].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

500 A Message of Hope
Song not found.
501 Morning Haze
Song not found.
502 White Waves
Song not found.
503 Midsummer Sun
No result found.
504 Cold Spring
No result found.
505 Halvmåne
Song not found.
506 Phaedra
Song not found.
507 Eyes of Redemption
Song not found.
508 Sogno Illuminum
No result found.
509 Endless Nights
No result found.
510 Lugnet
Song not found.
511 Ethereal
Song not found.
512 Spaces
Song not found.
513 Pampus
No result found.
514 Rêvasseur
Song not found.
515 Forever Vanished
Song not found.
516 Warmest Embrace
Song not found.
517 Ocean Deep
Song not found.
518 Phaedra
Song not found.
519 Will you be there when I wake up
No result found.
520 System
521 HOLIDAY
522 two queens in a king sized bed
523 How Low Can You Go
524 A Dreamer's Holiday - Spotify Singles
525 I Want To Break Free
526 It's Raining Men - Single Version
Song not found.
527 Born This Way
528 Don't Go Breaking My Heart
529 I'm Coming Out
530 angel rock
531 The Fall
532 Gospel For A New Century
533 Me

600

In [10]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[600:700].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

600 Silent Night - Piano Version
601 We'll Meet Again
Song not found.
602 My Friend the Forest
603 Sussex Carol (On Christmas Night) [Live]
Song not found.
604 In the Bleak Midwinter
Song not found.
605 The Twelve Days Of Christmas
Song not found.
606 I Wonder as I Wander
Song not found.
607 I Pray - Arr. Paul Campbell
No result found.
608 Now May We Singen
No result found.
609 The holly and the ivy
Song not found.
610 Ding Dong! Merrily on High
Song not found.
611 Once In Royal David's City (Desc. Stephen Cleobury)
No result found.
612 The First Nowell
Song not found.
613 Somewhere in My Memory - Chorus & Orchestra Version
Song not found.
614 Dappled Light
Song not found.
615 Fracture
616 Blow The Wind Southerly (Arr. Kanneh-Mason)
No result found.
617 These Days - Solo Piano Version
Song not found.
618 All Human Beings (Voiceless Mix) - Pt. 3
Song not found.
619 Blue Train - Remastered 2003
Song not found.
620 It Could Happen To You
621 Joy Spring
Song not found.
622 Weaver Of Dreams

700

In [11]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[700:800].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

700 Cedar Waltzin' ~ Don't You Worry 'bout a Thing
701 Captain’s Song
Song not found.
702 L.A. 58
Song not found.
703 All Things Considered
Song not found.
704 anywhere
705 Ophelia
706 Power Over Me
707 The Girl
708 Georgia
709 Ocean
Song not found.
710 That Something
Song not found.
711 Moments of Clarity
Song not found.
712 Train 201
Song not found.
713 Taormina
Song not found.
714 La del bar
715 Mar Y Cielo
Song not found.
716 Lo Mismo Que Usted
Song not found.
717 Calaveras y Diablitos
718 Oye Como Va
Song not found.
719 Hungry Eyes - From "Dirty Dancing" Soundtrack
No result found.
720 Hard to Say I'm Sorry - 2007 Remaster
Song not found.
721 Every Breath You Take
722 Heaven
723 All Out of Love
724 Iris
725 Torn - Acoustic MTV Unplugged
726 Crash into Me
727 Fade Into You
728 I Don't Want to Miss a Thing - From "Armageddon" Soundtrack
No result found.
729 Slow Down
730 You Got It
731 Dead Man Walking
732 I Like
733 Opia
734 Forever After All
735 The Bones
736 The Good Ones
737 Got

800

In [12]:
# Add the lyrics of the next 100 songs
t0 = time.time()
for index, row in song_artist_category_df[800:900].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

800 Cuando Me Digas
801 A la Orden del Flaco
No result found.
802 Mis Superpoderes
No result found.
803 Pensando En Verte
No result found.
804 La Captura III
No result found.
805 El Seriecito
No result found.
806 A la Orden del Flaco
No result found.
807 Plomo Traigo - en Vivo
808 El Perro
809 Cross Road Blues
810 Old devils
811 Seven Hells
812 The High Price of Inspiration
813 Crossroads
814 Restless Sinner
815 The Devil Wears a Suit and Tie
816 Me and the Devil Blues
817 Graveyard
818 Devil's Gonna Git You
819 Devil's Resting Place
820 Bad Things
821 Evil Devil Woman Blues
No result found.
822 Blue Guitar
823 Blue Guitar Stomp
Song not found.
824 Blue Guitar
Song not found.
825 Evil Woman
826 I Put A Spell On You
827 Don't You Lie To Me (I Get Evil) - 2002 Digital Remaster
No result found.
828 Standing At The Crossroads
829 Evil Is Going On
830 Sweet Love and Evil Women
831 Cross And Evil Woman Blues
832 Evil Hearted Woman - Alternate Take
833 Ain't No Grave (Sparse)
Song not found.


900

In [13]:
# Add the lyrics of the last 34 songs
t0 = time.time()
for index, row in song_artist_category_df[900:].iterrows():
    print(index, row['song'])
    song = row['song']
    artist = row['artist']
    query_url = base_url + f'/search/?q={song}%20{artist}'
    results = requests.get(query_url, headers=headers).json()
    try: 
        full_title = results['response']['hits'][0]['result']['full_title']
        full_title = full_title.replace('\u200b', '').replace('\xa0', ' ')
        song = re.sub('\(.*', '', song)
        if (song in full_title) or (artist in full_title):
            api_path = results['response']['hits'][0]['result']['api_path']
            song_url = base_url + api_path
            response = requests.get(song_url, headers={
                'Authorization': f'Bearer {genius_token}'}).json()
            path = response['response']['song']['path']
            page_url = 'http://genius.com' + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
            # Remove script tags from the lyrics
            [h.extract() for h in html('script')]
            lyrics = html.find('div', class_='lyrics').get_text()
            lyrics_list.append(lyrics)
        else:
            lyrics_list.append(np.nan)
            print('Song not found.')
    except: 
        lyrics_list.append(np.nan)
        print('No result found.')
t1 = time.time()
print(t1-t0)
len(lyrics_list)

900 JUMPING SHIP
901 Catch A Vibe
902 Hasara
903 Emakhaya (feat. Da Capo & Sun-El Musician)
904 Sghubu Ses Excellent (feat. Madumane, MDU aka TRP & Bongza)
No result found.
905 Ulazi
906 Phone Yam
Song not found.
907 Sithi Sithi
Song not found.
908 Uthando
Song not found.
909 Black Lungs
910 The Path
911 Protect The Land
Song not found.
912 Desperate Pleasures
913 Let The Chaos Reign
914 Black Lungs
915 Glass Heart
916 Lost Cause
917 Wearing Thin
918 Bulletproof
Song not found.
919 Rollin' (Air Raid Vehicle)
920 Butterfly
921 One Step Closer
922 Chop Suey!
923 Falling Away from Me
924 Unsainted
925 Uncontrollable Descent
926 Paramount
927 One Folk
928 Outsider
929 Paranoid
930 Immigrant Song - Remaster
No result found.
931 Ace of Spades
932 Highway Star - Remastered 2012
Song not found.
933 Holy Diver
934 Budum
935 Lighter
936 Call Me If
937 Lockdown
938 Gal Policy
939 Ocean Wave
940 Breaking News
941 Hot
942 Seen It Before
943 Promise
944 Tell Me What a Gwan
No result found.
945 So Da

959

In [27]:
# Add a column of lyrics to the DataFrame
song_artist_category_df['lyrics'] = lyrics_list
song_artist_category_df

,song,artist,artist_id,category,genres,filterd_genres,lyrics
0,positions,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['dance', 'pop', 'post-teen']","['dance', 'pop']",\n\n[Verse 1]\nHeaven sent you to me\nI'm just...
1,Levitating (feat. DaBaby),Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['uk', 'dance', 'pop']","['uk', 'dance', 'pop']",NaN
2,Therefore I Am,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],\n\n[Chorus]\nI'm not your friend\nOr anything...
3,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['dance', 'viral', 'pop', 'post-teen', 'canadi...","['dance', 'viral', 'pop', 'canadian']",\n\n[Verse 1: Shawn Mendes]\nYou put me on a p...
4,HOLIDAY,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['queer', 'lgbtq+', 'pop', 'country']","['pop', 'country']","\n\n[Intro]\nT-T-Tay Keith, Took it to ten (He..."
...,...,...,...,...,...,...,...
954,Punanny Medley,Elephant Man,6NOvBZrkd83MSD51xkq4on,caribbean,"['reggae', 'soca', 'dancehall', 'old']","['reggae', 'dancehall']",NaN
955,Bam Bam,Chaka Demus & Pliers,6RQkaOWddQmiLLJqSgnTbm,caribbean,"['funana', 'lovers', 'reggae']",['reggae'],"\n\nAaaaaahhh\nI want you to know that, I am t..."
956,Sweat (A La La La La Long),Inner Circle,5os0Ltvz8Q8BvXOPOd1frx,caribbean,"['reggae', 'roots']","['reggae', 'roots']",\n\n[Chorus]\nI've been watching you\nA La La ...
957,Beautiful,Damian Marley,3QJzdZJYIAcoET1GcfpNGi,caribbean,"['reggae', 'modern']","['reggae', 'modern']",\n\n[Verse 1: Damian Marley]\nAnd me and she w...


In [41]:
# Drop songs without lyrics
song_artist_category_df = song_artist_category_df.dropna()

In [42]:
# Clean the lyrics column of the DataFrame
song_artist_category_df['lyrics'] = song_artist_category_df['lyrics'].str.replace('\u205f', ' ').replace('  ', ' ').replace('  ', ' ')
song_artist_category_df['lyrics'] = song_artist_category_df['lyrics'].str.replace('\n', ' ').replace('-', ' ')
song_artist_category_df['lyrics'] = song_artist_category_df['lyrics'].str.replace('?', '').replace(',', '')
song_artist_category_df['lyrics'] = song_artist_category_df['lyrics'].str.replace('(', '').replace(')', '')
song_artist_category_df['lyrics'] = song_artist_category_df['lyrics'].str.replace("'", '').replace('’', '')
song_artist_category_df['lyrics'] = song_artist_category_df['lyrics'].str.replace('\[(.*?)\]', '', regex=True)
song_artist_category_df.head()

,song,artist,artist_id,category,genres,filterd_genres,lyrics
0,positions,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['dance', 'pop', 'post-teen']","['dance', 'pop']",Heaven sent you to me Im just hopin’ I dont...
2,Therefore I Am,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],"Im not your friend Or anything, damn You th..."
3,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['dance', 'viral', 'pop', 'post-teen', 'canadi...","['dance', 'viral', 'pop', 'canadian']",You put me on a pedestal and tell me Im the...
4,HOLIDAY,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['queer', 'lgbtq+', 'pop', 'country']","['pop', 'country']","T-T-Tay Keith, Took it to ten Hey) Ayy, i..."
5,On Me,Lil Baby,5f7VJjfbwm532GiveGC0ZK,toplists,"['rap', 'atl']","['rap', 'atl']","Whats happenin, Chi Chi Fill the bando up..."


In [33]:
# Find the number of songs without lyrics
song_artist_category_df['lyrics'].isnull().sum()

340